In [6]:
import torch
import numpy as np
import faiss
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from sentence_transformers import SentenceTransformer, CrossEncoder
from peft import PeftModel
from rank_bm25 import BM25Okapi
import nltk
from nltk.tokenize import sent_tokenize
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
import re
from rouge_score import rouge_scorer
import nltk
nltk.download('punkt_tab')
nltk.download('punkt', quiet=True)


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\arist\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

### Load datasets + prepare evaluation set

In [2]:
# Load corpus for retrieval (same as before)
ds_corpus = load_dataset("rag-datasets/rag-mini-wikipedia", "text-corpus")
passages_ds = ds_corpus["passages"]
original_texts = [ex["passage"] for ex in passages_ds]

# Load QA dataset for evaluation (we'll use this as gold standard)
ds_qa = load_dataset("rag-datasets/rag-mini-wikipedia", "question-answer")
qa_test = ds_qa["test"].select(range(100))  # 100 examples for eval

print(f"Corpus: {len(original_texts)} passages")
print(f"Eval QA: {len(qa_test)} questions")
print("Sample QA:", qa_test[0])


Corpus: 3200 passages
Eval QA: 100 questions
Sample QA: {'question': 'Was Abraham Lincoln the sixteenth President of the United States?', 'answer': 'yes', 'id': 0}


###  Advanced Chunking Strategies

In [3]:
def chunk_fixed_size(texts, chunk_size=200, overlap=50):
    """Fixed-size chunking with overlap"""
    chunks = []
    for text in texts:
        words = text.split()
        for i in range(0, len(words), chunk_size - overlap):
            chunk = " ".join(words[i:i+chunk_size])
            chunks.append(chunk)
    return chunks

def chunk_by_sentences(texts, max_sentences=5):
    """Chunk by sentence boundaries"""
    chunks = []
    for text in texts:
        sentences = sent_tokenize(text)
        for i in range(0, len(sentences), max_sentences):
            chunk = " ".join(sentences[i:i+max_sentences])
            chunks.append(chunk)
    return chunks

def chunk_semantic(texts, model_e, threshold=0.7):
    """Simple semantic chunking (group similar sentences)"""
    all_chunks = []
    for text in texts:
        sentences = sent_tokenize(text)
        if len(sentences) <= 3:
            all_chunks.append(text)
            continue
            
        sent_embs = model_e.encode(sentences, convert_to_numpy=True)
        chunks = []
        current_chunk = [sentences[0]]
        
        for i in range(1, len(sentences)):
            sim = cosine_similarity([sent_embs[0]], [sent_embs[i]])[0][0]
            if sim > threshold:
                current_chunk.append(sentences[i])
            else:
                chunks.append(" ".join(current_chunk))
                current_chunk = [sentences[i]]
        if current_chunk:
            chunks.append(" ".join(current_chunk))
        all_chunks.extend(chunks)
    return all_chunks

print("Chunking strategies ready!")


Chunking strategies ready!


### Load Embedding Models + BM25

In [4]:
# Dense retriever
embed_name = "sentence-transformers/all-MiniLM-L6-v2"
model_e = SentenceTransformer(embed_name)

# Reranker (cross-encoder)
reranker_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"
reranker = CrossEncoder(reranker_name)

print("Dense + Reranker loaded")


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 723.58it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
c:\Users\arist\miniconda3\envs\random\lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\arist\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details,

Dense + Reranker loaded


### Build Multiple Indexes (Chunking Comparison)

In [7]:
def build_indexes(texts_list, model_e):
    """Build FAISS + BM25 indexes for different chunking strategies"""
    indexes = {}
    
    for name, texts in texts_list.items():
        print(f"Indexing {name}: {len(texts)} chunks...")
        
        # Dense embeddings + FAISS
        embs = model_e.encode(texts, batch_size=64, show_progress_bar=True, normalize_embeddings=True)
        dim = embs.shape[1]
        faiss_idx = faiss.IndexFlatIP(dim)
        faiss_idx.add(embs)
        
        # BM25 (lexical)
        tokenized_texts = [nltk.word_tokenize(t.lower()) for t in texts]
        bm25 = BM25Okapi(tokenized_texts)
        
        indexes[name] = {
            'texts': texts,
            'faiss': faiss_idx,
            'bm25': bm25,
            'tokenized': tokenized_texts,
            'embs': embs
        }
    return indexes

# Create different chunking strategies
chunk_strategies = {
    'original': original_texts[:1000],  # Use subset for speed
    'fixed_200': chunk_fixed_size(original_texts[:500], 200, 50),
    'sentences': chunk_by_sentences(original_texts[:500]),
}

# Build all indexes
indexes = build_indexes(chunk_strategies, model_e)
print("All indexes built!")


Indexing original: 1000 chunks...


Batches: 100%|██████████| 16/16 [00:01<00:00, 10.67it/s]


Indexing fixed_200: 538 chunks...


Batches: 100%|██████████| 9/9 [00:00<00:00, 27.34it/s]


Indexing sentences: 596 chunks...


Batches: 100%|██████████| 10/10 [00:00<00:00, 29.36it/s]


All indexes built!


### Advanced Multi-stage Retrieval

In [12]:
def hybrid_search(query, index_data, k_final=5, k_bm25=50, k_dense=20):
    """BM25 → Dense → Rerank pipeline (FIXED)"""
    texts, faiss_idx, bm25, tokenized = index_data['texts'], index_data['faiss'], index_data['bm25'], index_data['tokenized']
    
    # Stage 1: BM25 (lexical)
    tokenized_query = nltk.word_tokenize(query.lower())
    bm25_scores = bm25.get_scores(tokenized_query)
    top_bm25_idx = np.argsort(bm25_scores)[-min(k_bm25, len(bm25_scores)):]  # 🛡️ Safety
    
    if len(top_bm25_idx) == 0:
        return []  # No candidates
    
    # Stage 2: Dense retrieval on BM25 candidates
    candidate_texts = [texts[i] for i in top_bm25_idx]
    q_emb = model_e.encode([query], normalize_embeddings=True)
    k_search = min(k_dense, len(top_bm25_idx))
    dense_scores, dense_idx = faiss_idx.search(q_emb, k_search)
    dense_idx = dense_idx[0]
    
    # 🛡️ FIX: Clamp dense_idx to valid BM25 candidates
    valid_dense_idx = []
    for doc_idx in dense_idx:
        if doc_idx < len(top_bm25_idx):
            valid_dense_idx.append(doc_idx)
        else:
            # Fallback to best BM25
            valid_dense_idx.append(0)
    
    # Combine scores (BM25 + dense)
    hybrid_scores = []
    for i, clamped_idx in enumerate(valid_dense_idx[:k_dense]):
        bm25_score = bm25_scores[top_bm25_idx[clamped_idx]]
        dense_score = dense_scores[0][i]
        hybrid_score = 0.4 * bm25_score + 0.6 * dense_score
        hybrid_scores.append((hybrid_score, candidate_texts[clamped_idx]))
    
    # Stage 3: Rerank top candidates
    top_hybrid = sorted(hybrid_scores, reverse=True)[:10]
    if len(top_hybrid) > 1:
        pairs = [(query, text) for _, text in top_hybrid]
        rerank_scores = reranker.predict(pairs)
        top_final = [(rerank_scores[i], text) for i, (_, text) in enumerate(top_hybrid)]
    else:
        top_final = top_hybrid
    
    return sorted(top_final, reverse=True)[:k_final]


### Improved RAG Prompting

In [9]:
def build_advanced_prompt(query, retrieved_passages, strategy_name=""):
    """Advanced RAG prompt with citation, refusal, and CoT"""
    ctx_text = "\n\n".join([f"[C{i+1}] {text[:400]}..." for i, (_, text) in enumerate(retrieved_passages)])
    
    prompt = f"""You are an expert research assistant. Answer using ONLY the provided context.

CONTEXT (retrieved with {strategy_name}):
{ctx_text}

QUESTION: {query}

INSTRUCTIONS:
1. Use ONLY information from the context above
2. If context lacks sufficient information, say "I don't know based on provided context"
3. Cite sources using [C1], [C2], etc. format
4. Provide a concise, accurate answer

ANSWER:"""
    
    return prompt

print("Advanced prompting ready!")


Advanced prompting ready!


### Reload Mistral LoRA + Pipeline

In [10]:
# Same as before but with better generation params
model_name = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=bnb_config, torch_dtype=torch.float16)
model = PeftModel.from_pretrained(base_model, "./mistral-rag-mini-lora")

gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,
    temperature=0.1,  # Lower for consistency
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.1,
)

def generate_answer(prompt):
    result = gen(prompt, return_full_text=False)[0]["generated_text"].strip()
    return result.split("ANSWER:")[-1].strip() if "ANSWER:" in result else result


`torch_dtype` is deprecated! Use `dtype` instead!
Loading weights: 100%|██████████| 291/291 [01:15<00:00,  3.86it/s, Materializing param=model.norm.weight]                              
Passing `generation_config` together with generation-related arguments=({'temperature', 'max_new_tokens', 'do_sample', 'repetition_penalty', 'top_p'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


### End-to-End Advanced RAG

In [13]:
def advanced_rag(query, indexes, k=5, show_comparison=False):
    """Compare all chunking strategies"""
    results = {}
    
    for strategy, index_data in indexes.items():
        # Multi-stage retrieval
        retrieved = hybrid_search(query, index_data, k_final=k)
        
        # Advanced prompt
        prompt = build_advanced_prompt(query, retrieved, strategy)
        
        # Generate
        answer = generate_answer(prompt)
        
        results[strategy] = {
            'retrieved': retrieved,
            'prompt': prompt,
            'answer': answer
        }
    
    if show_comparison:
        print(f"\n{'='*80}")
        print(f"QUERY: {query}")
        print(f"{'='*80}")
        for strategy, res in results.items():
            print(f"\n{strategy.upper()} (top-1 retrieval score: {res['retrieved'][0][0]:.3f})")
            print(f"CONTEXT: {res['retrieved'][0][1][:200]}...")
            print(f"ANSWER: {res['retrieved'][:100]}")
            print("-" * 50)
    
    return results

# Test it!
query = "Who was Ulysses S. Grant?"
results = advanced_rag(query, indexes, k=3, show_comparison=True)


Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



QUERY: Who was Ulysses S. Grant?

ORIGINAL (top-1 retrieval score: -9.232)
CONTEXT: Blaise Pascal ( ), (June 19 1623   August 19 1662) was a French mathematician, physicist, and religious philosopher. He was a child prodigy who was educated by his father. Pascal's earliest work was i...
ANSWER: [(np.float32(-9.231781), "Blaise Pascal ( ), (June 19 1623   August 19 1662) was a French mathematician, physicist, and religious philosopher. He was a child prodigy who was educated by his father. Pascal's earliest work was in the natural and applied sciences where he made important contributions to the construction of mechanical calculators, the study of fluids, and clarified the concepts of pressure and vacuum by generalizing the work of Evangelista Torricelli. Pascal also wrote in defense of the scientific method."), (np.float32(-9.231781), "Blaise Pascal ( ), (June 19 1623   August 19 1662) was a French mathematician, physicist, and religious philosopher. He was a child prodigy who was edu

### Comprehensive Evaluation Metrics

In [14]:
def normalize_answer(s):
    """Normalize for exact match"""
    return re.sub(r'\s+', ' ', s.lower().strip())

def exact_match(pred, gold):
    return normalize_answer(pred) == normalize_answer(gold)

def rouge_score(pred, gold):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(gold, pred)
    return scores['rouge1'].fmeasure, scores['rougeL'].fmeasure

def evaluate_retrieval(qa_example, retrieved_passages, original_texts):
    """Does retrieval find relevant passages?"""
    gold_answer = qa_example['answer'].lower()
    relevant_chunks = 0
    total_chunks = len(retrieved_passages)
    
    for _, passage in retrieved_passages:
        if any(word in passage.lower() for word in gold_answer.split() if len(word) > 3):
            relevant_chunks += 1
    
    recall = relevant_chunks / max(total_chunks, 1)
    return recall

def full_evaluation(indexes, qa_test, n_eval=20):
    """Run full evaluation across strategies"""
    results = defaultdict(list)
    
    for i in range(n_eval):
        qa = qa_test[i]
        query = qa['question']
        gold = qa['answer']
        
        strategy_results = advanced_rag(query, indexes, k=3, show_comparison=False)
        
        for strategy, res in strategy_results.items():
            pred = res['answer']
            
            # QA metrics
            em = exact_match(pred, gold)
            r1, rL = rouge_score(pred, gold)
            
            # Retrieval metric
            recall = evaluate_retrieval(qa, res['retrieved'], original_texts)
            
            results[strategy].append({
                'em': em,
                'rouge1': r1,
                'rougeL': rL,
                'recall': recall
            })
    
    # Aggregate
    print("\n" + "="*80)
    print("EVALUATION RESULTS (across strategies)")
    print("="*80)
    
    eval_table = []
    for strategy, metrics in results.items():
        avg_em = np.mean([m['em'] for m in metrics])
        avg_r1 = np.mean([m['rouge1'] for m in metrics])
        avg_rL = np.mean([m['rougeL'] for m in metrics])
        avg_recall = np.mean([m['recall'] for m in metrics])
        
        eval_table.append([strategy, f"{avg_em:.1%}", f"{avg_r1:.3f}", f"{avg_rL:.3f}", f"{avg_recall:.3f}"])
        print(f"{strategy:12}: EM={avg_em:.1%} | R1={avg_r1:.3f} | RL={avg_rL:.3f} | Recall={avg_recall:.3f}")
    
    return eval_table

# Run evaluation!
eval_results = full_evaluation(indexes, qa_test)


Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both


EVALUATION RESULTS (across strategies)
original    : EM=55.0% | R1=0.659 | RL=0.659 | Recall=0.000
fixed_200   : EM=50.0% | R1=0.592 | RL=0.592 | Recall=0.050
sentences   : EM=45.0% | R1=0.536 | RL=0.536 | Recall=0.000


In [15]:
## Cell 11 – Merouane External Knowledge Test (RAG override)

# 1. Inject external fact into ONE chunking strategy
merouane_fact = "The creator of this RAG notebook is named Merouane. Merouane built the fine-tuning pipeline and advanced retrieval system."

# Add to 'fixed_200' strategy only (to compare with others)
indexes['merouane_fixed'] = indexes['fixed_200'].copy()
indexes['merouane_fixed']['texts'].append(merouane_fact)

# Rebuild index with new fact
new_emb = model_e.encode([merouane_fact], normalize_embeddings=True, convert_to_numpy=True)
indexes['merouane_fixed']['faiss'].add(new_emb)
indexes['merouane_fixed']['tokenized'].append(nltk.word_tokenize(merouane_fact.lower()))

print("✅ Merouane fact injected into 'merouane_fixed' index")
print(f"New index size: {indexes['merouane_fixed']['faiss'].ntotal}")


✅ Merouane fact injected into 'merouane_fixed' index
New index size: 539


In [20]:
## Test the override
merouane_query = "Who created this RAG notebook?"

# Compare ALL strategies (including the one with Merouane)
results = advanced_rag(merouane_query, indexes, k=3, show_comparison=True)

print("\n" + "="*100)
print("🎯 KEY TEST: Does RAG override model knowledge?")
print("="*100)

for strategy, res in results.items():
    top_context = res['retrieved'][0][1]
    has_merouane = "Merouane" in top_context
    
    print(f"{strategy:15} | Top context has Merouane: {has_merouane}")
    print(f"                  | Context preview: {top_context[:100]}...")
    print(f"                  | Answer cites it: {'Merouane' in res['answer']}")
    print()


Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



QUERY: Who created this RAG notebook?

ORIGINAL (top-1 retrieval score: -11.264)
CONTEXT: In his work on static electricity, Faraday demonstrated that the charge only resided on the exterior of a charged conductor, and exterior charge had no influence on anything enclosed within a conducto...
ANSWER: [(np.float32(-11.26402), 'In his work on static electricity, Faraday demonstrated that the charge only resided on the exterior of a charged conductor, and exterior charge had no influence on anything enclosed within a conductor. This is because the exterior charges redistribute such that the interior fields due to them cancel. This shielding effect is used in what is now known as a Faraday cage.'), (np.float32(-11.26402), 'In his work on static electricity, Faraday demonstrated that the charge only resided on the exterior of a charged conductor, and exterior charge had no influence on anything enclosed within a conductor. This is because the exterior charges redistribute such that the int

In [18]:
## Cell 12 – DEBUG SAFE: Merouane retrieval (FAISS-proof)

def debug_merouane_search(query, index_data):
    """Debug: search with FULL safety bounds"""
    texts = index_data['texts']
    faiss_idx = index_data['faiss']
    bm25 = index_data['bm25']
    
    # 1. Find Merouane position
    merouane_pos = next((i for i, t in enumerate(texts) if "Merouane" in t), None)
    print(f"🔍 DEBUG MEROUANE")
    print(f"Index size: {len(texts)} chunks")
    print(f"Merouane fact at exact position: {merouane_pos}")
    
    if merouane_pos is None:
        print("❌ Merouane fact NOT found in texts!")
        return
    
    # 2. BM25 top 100 (SAFE)
    tokenized_query = nltk.word_tokenize(query.lower())
    bm25_scores = bm25.get_scores(tokenized_query)
    top_bm25_idx = np.argsort(bm25_scores)[-min(100, len(bm25_scores)):]
    
    # 3. FULL FAISS search on ALL index (not just BM25 candidates)
    q_emb = model_e.encode([query], normalize_embeddings=True)
    dense_scores, dense_idx = faiss_idx.search(q_emb, 20)
    
    print(f"\n📊 Top 20 DENSE scores (across ALL chunks):")
    merouane_in_top20 = False
    for i in range(min(20, len(dense_idx[0]))):
        global_idx = dense_idx[0][i]  # Index dans l'index FAISS complet
        if global_idx >= len(texts):
            print(f"  {i+1:2d}. INVALID IDX {global_idx} (skip)")
            continue
            
        score = dense_scores[0][i]
        text_preview = texts[global_idx][:100]
        is_merouane = global_idx == merouane_pos
        
        print(f"  {i+1:2d}. Score={score:6.3f} | Merouane: {is_merouane} | {text_preview}...")
        if is_merouane:
            merouane_in_top20 = True
    
    print(f"\n🎯 Merouane in top-20 dense? {merouane_in_top20}")

# Run debug
debug_merouane_search("Who created this RAG notebook?", indexes['merouane_fixed'])


🔍 DEBUG MEROUANE
Index size: 539 chunks
Merouane fact at exact position: 538

📊 Top 20 DENSE scores (across ALL chunks):
   1. Score= 0.657 | Merouane: True | The creator of this RAG notebook is named Merouane. Merouane built the fine-tuning pipeline and adva...
   2. Score= 0.347 | Merouane: False | An early, undated photograph of Grover Cleveland from the Cleveland Family Papers at the New Jersey ...
   3. Score= 0.292 | Merouane: False | * Gore Vidal. Lincoln ISBN 0-375-70876-6, a novel....
   4. Score= 0.292 | Merouane: False | * Sobel, Robert, Coolidge: An American Enigma (1998), ISBN 0895264102....
   5. Score= 0.291 | Merouane: False | Faraday's greatest work was probably with electricity and magnetism. The first experiment which he r...
   6. Score= 0.284 | Merouane: False | As a chemist, Faraday discovered benzene, investigated the clathrate hydrate of chlorine, invented a...
   7. Score= 0.284 | Merouane: False | * Williams, L. Pearce (1971), Faraday: A Biography, Simon and S

In [19]:
## Cell 13 – FORCE Merouane test (sans hybrid complexité)

def simple_merouane_test():
    index_data = indexes['merouane_fixed']
    query = "Who created this RAG notebook?"
    
    # Direct FAISS search (k=50, pas de BM25)
    q_emb = model_e.encode([query], normalize_embeddings=True)
    scores, indices = index_data['faiss'].search(q_emb, 50)
    
    # Find Merouane
    merouane_chunks = []
    for i, idx in enumerate(indices[0]):
        if idx < len(index_data['texts']):
            text = index_data['texts'][idx]
            if "Merouane" in text:
                merouane_chunks.append((scores[0][i], text))
                print(f"✅ MEROUANE trouvé! Position {i+1}/50, score={scores[0][i]:.3f}")
                break
    
    if merouane_chunks:
        # Build prompt avec Merouane
        ctx = "\n\n".join([f"[C{i+1}] {t[:300]}..." for i, (_, t) in enumerate(merouane_chunks)])
        prompt = f"""CONTEXT:
{ctx}

QUESTION: {query}
ANSWER using ONLY context:"""
        
        answer = generate_answer(prompt)
        print(f"\n🤖 MISTRAL ANSWER: {answer}")
        print("✅ RAG INJECTION SUCCESS!")
    else:
        print("❌ Merouane not in top-50 dense")

simple_merouane_test()


Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


✅ MEROUANE trouvé! Position 1/50, score=0.657

🤖 MISTRAL ANSWER: Merouane
✅ RAG INJECTION SUCCESS!


In [22]:
## Cell 14 – Test FINAL: Toutes stratégies avec k=20

merouane_query = "Who created this RAG notebook?"

# Test TOUTES les stratégies avec k=20 (pour attraper Merouane)
results_k20 = advanced_rag(merouane_query, indexes, k=20, show_comparison=False)

print("="*100)
print("🎯 TEST FINAL: RAG Override (k=20 pour toutes stratégies)")
print("="*100)

for strategy, res in results_k20.items():
    # Vérif top-5 (Merouane devrait être dedans avec k=20)
    top_contexts = [t[1] for t in res['retrieved'][:5]]
    has_merouane = any("Merouane" in ctx for ctx in top_contexts)
    merouane_pos = next((i+1 for i, (_, ctx) in enumerate(res['retrieved']) if "Merouane" in ctx), "N/A")
    
    print(f"{strategy:15} | Merouane rank: {merouane_pos:2}/{len(res['retrieved'])} | In top-5: {has_merouane}")
    print(f"                  | Top-1 preview: {res['retrieved'][0][1][:100]}...")
    print(f"                  | Answer: {'Merouane' in res['answer']} → {res['answer'][:80]}...")
    print("-" * 80)


Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🎯 TEST FINAL: RAG Override (k=20 pour toutes stratégies)
original        | Merouane rank: N/A/10 | In top-5: False
                  | Top-1 preview: In his work on static electricity, Faraday demonstrated that the charge only resided on the exterior...
                  | Answer: False → Michael Faraday...
--------------------------------------------------------------------------------
fixed_200       | Merouane rank: N/A/10 | In top-5: False
                  | Top-1 preview: Born in Clermont-Ferrand, in the Auvergne region of France, Blaise Pascal lost his mother, Antoinett...
                  | Answer: False → Blaise Pascal...
--------------------------------------------------------------------------------
sentences       | Merouane rank: N/A/10 | In top-5: False
                  | Top-1 preview: McClellan, a lifelong Democrat who was temperamentally conservative, was relieved as general-in-chie...
                  | Answer: False → George B. McClellan...
-----------------------

In [23]:
## Cell 16 – VÉRIF + RE-INJECT Merouane (100% sûr)

print("🔍 VÉRIFICATION des indexes:")
for name, data in indexes.items():
    texts = data['texts']
    merouane_count = sum(1 for t in texts if "Merouane" in t)
    print(f"{name:15} | Chunks total: {len(texts):4d} | Merouane chunks: {merouane_count}")

# REBUILD merouane_fixed proprement
merouane_fact = "The creator of this RAG notebook is named Merouane. Merouane built the fine-tuning pipeline and advanced retrieval system."

# Copie + ajout PROPRE
new_texts = indexes['fixed_200']['texts'].copy()
new_texts.append(merouane_fact)

new_embs = model_e.encode(new_texts, normalize_embeddings=True, batch_size=64)
dim = new_embs.shape[1]
new_faiss = faiss.IndexFlatIP(dim)
new_faiss.add(new_embs)

new_tokenized = [nltk.word_tokenize(t.lower()) for t in new_texts]
new_bm25 = BM25Okapi(new_tokenized)

indexes['merouane_fixed_v2'] = {
    'texts': new_texts,
    'faiss': new_faiss,
    'bm25': new_bm25,
    'tokenized': new_tokenized,
    'embs': new_embs
}

print(f"\n✅ REBUILD merouane_fixed_v2:")
print(f"  New size: {len(new_texts)} chunks")
print(f"  Merouane at: {new_texts.index(merouane_fact)}")
print(f"  FAISS ntotal: {new_faiss.ntotal}")


🔍 VÉRIFICATION des indexes:
original        | Chunks total: 1000 | Merouane chunks: 0
fixed_200       | Chunks total:  539 | Merouane chunks: 1
sentences       | Chunks total:  596 | Merouane chunks: 0
merouane_fixed  | Chunks total:  539 | Merouane chunks: 1

✅ REBUILD merouane_fixed_v2:
  New size: 540 chunks
  Merouane at: 538
  FAISS ntotal: 540


In [24]:
## Cell 17 – Test AVEC merouane_fixed_v2

merouane_query = "Who created this RAG notebook?"
test_indexes = {k: v for k, v in indexes.items() if k != 'merouane_fixed'}
test_indexes['merouane_fixed_v2'] = indexes['merouane_fixed_v2']

results_final = advanced_rag(merouane_query, test_indexes, k=20, show_comparison=False)

print("="*100)
print("🎯 TEST FINAL v2 (avec rebuild)")
print("="*100)

for strategy, res in results_final.items():
    merouane_pos = next((i+1 for i, (_, ctx) in enumerate(res['retrieved']) if "Merouane" in ctx), "N/A")
    has_merouane_top5 = any("Merouane" in ctx[1] for ctx in res['retrieved'][:5])
    
    print(f"{strategy:15} | Merouane rank: {merouane_pos:3} | Top5: {has_merouane_top5}")
    print(f"                  | Answer: {'✅ Merouane cité!' if 'Merouane' in res['answer'] else '❌ Pas trouvé'}")
    print(f"                  | '{res['answer'][:60]}...'")
    print("-" * 80)


Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🎯 TEST FINAL v2 (avec rebuild)
original        | Merouane rank: N/A | Top5: False
                  | Answer: ❌ Pas trouvé
                  | 'Michael Faraday...'
--------------------------------------------------------------------------------
fixed_200       | Merouane rank: N/A | Top5: False
                  | Answer: ❌ Pas trouvé
                  | 'Blaise Pascal...'
--------------------------------------------------------------------------------
sentences       | Merouane rank: N/A | Top5: False
                  | Answer: ❌ Pas trouvé
                  | 'The RAG...'
--------------------------------------------------------------------------------
merouane_fixed_v2 | Merouane rank: N/A | Top5: False
                  | Answer: ❌ Pas trouvé
                  | 'Abraham Lincoln...'
--------------------------------------------------------------------------------


In [25]:
## Cell 18 – TEST DIRECT FAISS (diag ultime)

merouane_query = "Who created this RAG notebook?"
index_data = indexes['merouane_fixed_v2']

# 1. Vérif fait présent
merouane_pos = next((i for i, t in enumerate(index_data['texts']) if "Merouane" in t), None)
print(f"✅ Merouane dans texts à position: {merouane_pos}")

# 2. Test DIRECT FAISS (pas de BM25)
q_emb = model_e.encode([merouane_query], normalize_embeddings=True)
print(f"Query embedding shape: {q_emb.shape}")

scores, indices = index_data['faiss'].search(q_emb, k=10)
print(f"FAISS search: scores={scores[0][:5]}, indices={indices[0]}")

# 3. Vérif chunks retrouvés
print("\nTop 5 chunks retrouvés:")
for i, idx in enumerate(indices[0][:5]):
    if idx < len(index_data['texts']):
        text = index_data['texts'][idx]
        is_merouane = "Merouane" in text
        print(f"  {i+1}. Score={scores[0][i]:6.3f} | Merouane: {is_merouane} | {text[:80]}...")
    else:
        print(f"  {i+1}. INVALID INDEX {idx}")

print(f"\n🎯 Merouane retrouvé? {'✅ OUI' if merouane_pos in indices[0][:10] else '❌ NON'}")


✅ Merouane dans texts à position: 538
Query embedding shape: (1, 384)
FAISS search: scores=[0.6573448  0.6573448  0.3474611  0.29217365 0.29168218], indices=[539 538 512 422 506  97  79 138 497 416]

Top 5 chunks retrouvés:
  1. Score= 0.657 | Merouane: True | The creator of this RAG notebook is named Merouane. Merouane built the fine-tuni...
  2. Score= 0.657 | Merouane: True | The creator of this RAG notebook is named Merouane. Merouane built the fine-tuni...
  3. Score= 0.347 | Merouane: False | An early, undated photograph of Grover Cleveland from the Cleveland Family Paper...
  4. Score= 0.292 | Merouane: False | * Gore Vidal. Lincoln ISBN 0-375-70876-6, a novel....
  5. Score= 0.292 | Merouane: False | * Sobel, Robert, Coolidge: An American Enigma (1998), ISBN 0895264102....

🎯 Merouane retrouvé? ✅ OUI


In [26]:
## Cell 19 – REBUILD MINI (5 chunks pour test)

mini_texts = [
    "Michael Faraday discovered electromagnetic induction.",
    "Blaise Pascal invented the calculator.",
    "Abraham Lincoln was US president.",
    "The creator of this RAG notebook is Merouane.",  # Position 3
    "George McClellan Civil War general."
]

mini_embs = model_e.encode(mini_texts, normalize_embeddings=True)
dim = mini_embs.shape[1]
mini_faiss = faiss.IndexFlatIP(dim)
mini_faiss.add(mini_embs)

mini_tokenized = [nltk.word_tokenize(t.lower()) for t in mini_texts]
mini_bm25 = BM25Okapi(mini_tokenized)

indexes['mini_test'] = {
    'texts': mini_texts,
    'faiss': mini_faiss,
    'bm25': mini_bm25,
    'tokenized': mini_tokenized
}

print("✅ Mini index créé (5 chunks, Merouane #3)")
print("Texts:", [t[:40]+"..." for t in mini_texts])


✅ Mini index créé (5 chunks, Merouane #3)
Texts: ['Michael Faraday discovered electromagnet...', 'Blaise Pascal invented the calculator....', 'Abraham Lincoln was US president....', 'The creator of this RAG notebook is Mero...', 'George McClellan Civil War general....']


In [27]:
## Test mini
mini_results = advanced_rag("Who created this RAG notebook?", {'mini_test': indexes['mini_test']}, k=5)
print("Mini test answer:", mini_results['mini_test']['answer'])


Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Mini test answer: Merouane


In [28]:
## Cell 20 – RAG PURE DENSE (sans BM25 cassé)

def pure_dense_rag(query, indexes, k=50):
    """RAG pure FAISS (debug version)"""
    results = {}
    for strategy, data in indexes.items():
        q_emb = model_e.encode([query], normalize_embeddings=True)
        scores, indices = data['faiss'].search(q_emb, k)
        
        retrieved = []
        for i in range(k):
            idx = indices[0][i]
            if idx < len(data['texts']):
                retrieved.append((float(scores[0][i]), data['texts'][idx]))
        
        prompt = build_advanced_prompt(query, retrieved[:10], strategy)
        answer = generate_answer(prompt)
        
        results[strategy] = {'retrieved': retrieved, 'answer': answer}
    
    return results

# Test pur dense
pure_results = pure_dense_rag("Who created this RAG notebook?", indexes, k=50)

print("="*100)
print("🎯 PURE DENSE RAG (k=50, sans BM25)")
print("="*100)

for strategy, res in pure_results.items():
    merouane_pos = next((i+1 for i, (_, t) in enumerate(res['retrieved']) if "Merouane" in t), "N/A")
    print(f"{strategy:15} | Merouane rank: {merouane_pos} | Answer: {res['answer'][:60]}...")


Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both

🎯 PURE DENSE RAG (k=50, sans BM25)
original        | Merouane rank: N/A | Answer: ...
fixed_200       | Merouane rank: 1 | Answer: Merouane...
sentences       | Merouane rank: N/A | Answer: Michael Faraday...
merouane_fixed  | Merouane rank: 1 | Answer: Merouane...
merouane_fixed_v2 | Merouane rank: 1 | Answer: Merouane...
mini_test       | Merouane rank: 1 | Answer: Merouane...
